Alexis Perumal, 5/4/20
HW22, Level 1 with help from Kevin Nguyen

In [0]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"


In [0]:
import findspark
findspark.init()

# New Section

In [4]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-05-07 22:06:13--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  3.44MB/s    in 0.3s    

2020-05-07 22:06:14 (3.44 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()


In [34]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
video_games_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True)
video_games_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [0]:
from pyspark.sql.types import IntegerType
video_games_df = video_games_df.withColumn("customer_id", video_games_df["customer_id"].cast(IntegerType()))
video_games_df = video_games_df.withColumn("product_parent", video_games_df["product_parent"].cast(IntegerType()))
video_games_df = video_games_df.withColumn("star_rating", video_games_df["star_rating"].cast(IntegerType()))
video_games_df = video_games_df.withColumn("helpful_votes", video_games_df["helpful_votes"].cast(IntegerType()))
video_games_df = video_games_df.withColumn("total_votes", video_games_df["total_votes"].cast(IntegerType()))

In [36]:
# Count the number of rows
video_games_df.count()

1785997

In [0]:
# For development purposes, limit the size of the dataframe so things go quicker!
# video_games_df = video_games_df.limit(1000)

In [38]:
# Count the number of rows
video_games_df.count()

1785997

In [39]:
# Drop null values
video_games_df = video_games_df.dropna()
video_games_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [40]:
# Count the number of rows
video_games_df.count()

1785886

In [0]:
# # Drop duplicates
# video_games_df = video_games_df.dropDuplicates(subset=['product_id'])
# video_games_df.show()
# video_games_df.count()

Note the schema of the four tables:

CREATE TABLE review_id_table (
  review_id TEXT PRIMARY KEY NOT NULL,
  customer_id INTEGER,
  product_id TEXT,
  product_parent INTEGER,
  review_date DATE -- this should be in the formate yyyy-mm-dd
);

-- This table will contain only unique values
CREATE TABLE products (
  product_id TEXT PRIMARY KEY NOT NULL UNIQUE,
  product_title TEXT
);

-- Customer table for first data set
CREATE TABLE customers (
  customer_id INT PRIMARY KEY NOT NULL UNIQUE,
  customer_count INT
);

-- vine table
CREATE TABLE vine_table (
  review_id TEXT PRIMARY KEY,
  star_rating INTEGER,
  helpful_votes INTEGER,
  total_votes INTEGER,
  vine TEXT
);


In [42]:
# Build the Table 1, in PySpark: review_id_table

from pyspark.sql.functions import to_date
review_id_df = video_games_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RTIS3L2M1F5SM|   12039526|B001CXYMFS|     737716809| 2015-08-31|
| R1ZV7R40OLHKD|    9636577|B00M920ND6|     569686175| 2015-08-31|
|R3BH071QLH8QMC|    2331478|B0029CSOD2|      98937668| 2015-08-31|
|R127K9NTSXA2YH|   52495923|B00GOOSV98|      23143350| 2015-08-31|
|R32ZWUXDJPW27Q|   14533949|B00Y074JOM|     821342511| 2015-08-31|
|R3AQQ4YUKJWBA6|    2377552|B002UBI6W6|     328764615| 2015-08-31|
|R2F0POU5K6F73F|   17521011|B008XHCLFO|      24234603| 2015-08-31|
|R3VNR804HYSMR6|   19676307|B00BRA9R6A|     682267517| 2015-08-31|
| R3GZTM72WA2QH|     224068|B009EPWJLA|     435241890| 2015-08-31|
| RNQOY62705W1K|   48467989|B0000AV7GB|     256572651| 2015-08-31|
|R1VTIA3JTYBY02|     106569|B00008KTNN|     384411423| 2015-08-31|
|R29DOU8791QZL8|   48269642|B000A3IA0Y|     472622859| 2015-08

In [43]:
# Build the Table 2, in PySpark: products table

# Extract products table info
products_df = video_games_df.select(["product_id", "product_title"]).dropDuplicates(subset=['product_id'])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0439394422|school house rock...|
|9861064222|Black Nintendo Wi...|
|B00000K1XN|        WWF Attitude|
|B00001OX40|   Neo Cherry Master|
|B00001QEQD|     USCF Chess - PC|
|B00001XDTO|        Yoda Stories|
|B00001ZWQ0|Legend of the Riv...|
|B0000296ZL|      Game Boy Color|
|B00002SFMS|     Boss Rally - PC|
|B00002SSYX|Ristar (Sega Game...|
|B00002STA7|    NFL Football '94|
|B00002STH3|           Extreme G|
|B00002STJC|                 MDK|
|B00002STWV|Virtual Open Tenn...|
|B00002SW1K|        World Heroes|
|B00002SW49|Lawnmower Man - S...|
|B000035XH1|RoboCop vs. Termi...|
|B000035XYM|Alien vs. Predato...|
|B000035Y3U|Top Gear 2 - Nint...|
|B000046S3Z|           Crystalis|
+----------+--------------------+
only showing top 20 rows



In [44]:
# Build the Table 3, in PySpark: customers table
import pyspark.sql.functions as sf

customers_df = video_games_df.groupBy("customer_id").agg(sf.count("product_id").alias("customer_count"))
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   48670265|             1|
|   49103216|             2|
|    1131200|             1|
|   43076447|             2|
|   46261368|             1|
|    4883305|             5|
|   41192649|             1|
|   40985731|             7|
|   10437900|             2|
|   22245671|             1|
|    2574873|             1|
|    4696154|             1|
|    5621202|             1|
|    5871933|             2|
|   44089812|             1|
|    2845910|             1|
|    5274369|             1|
|   39069693|             2|
|     137793|             1|
|   31914942|             3|
+-----------+--------------+
only showing top 20 rows



In [45]:
# Build the Table 4, in PySpark: vine_table

vine_table_df = video_games_df.select(["review_id", "star_rating", "helpful_votes", "total_votes",
                                       "vine"])
vine_table_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RTIS3L2M1F5SM|          5|            0|          0|   N|
| R1ZV7R40OLHKD|          5|            0|          0|   N|
|R3BH071QLH8QMC|          1|            0|          1|   N|
|R127K9NTSXA2YH|          3|            0|          0|   N|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|
|R2F0POU5K6F73F|          5|            0|          0|   N|
|R3VNR804HYSMR6|          5|            0|          0|   N|
| R3GZTM72WA2QH|          5|            0|          0|   N|
| RNQOY62705W1K|          4|            0|          0|   N|
|R1VTIA3JTYBY02|          5|            0|          0|   N|
|R29DOU8791QZL8|          1|            0|          0|   N|
|R15DUT1VIJ9RJZ|          2|            0|          0|   N|
|R3IMF2MQ3OU9ZM|          4|            

Push to RDS

In [0]:
 # Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.ccavomop9z5f.us-east-1.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"postgres", 
          "password": "", 
          "driver":"org.postgresql.Driver"}

In [0]:
# Table 1
# Write DataFrame to review_id_table table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [48]:
review_id_df.head()

Row(review_id='RTIS3L2M1F5SM', customer_id=12039526, product_id='B001CXYMFS', product_parent=737716809, review_date=datetime.date(2015, 8, 31))

In [0]:
# Table 2
# Write DataFrame to products table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [0]:
# Table 3
# Write DataFrame to customers table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [0]:
# Table 4
# Write DataFrame to vine_table table in RDS
vine_table_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

In [52]:
# products_df.filter(products_df.product_id =='0001714422').show()
products_df.filter(products_df.product_id =='0439394422').show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0439394422|school house rock...|
+----------+--------------------+



In [53]:
products_df.head()

Row(product_id='0439394422', product_title='school house rock america rock')